In [17]:
import pandas as pd
import math


def fill_missing_intervals(df):
    df = df.sort_values(by='start_frame')

    dfs = []

    for i in range(len(df) - 1):
        current_row = df.loc[i]
        next_row = df.loc[i + 1]

        missing_interval_start = current_row['stop_frame'] + 1
        missing_interval_end = next_row['start_frame'] - 1

        if missing_interval_start <= missing_interval_end:
            new_row = pd.DataFrame({
                'uid': 90,
                'participant_id': 'S04',
                'video_id': 'S04_01',
                'narration': 'None',
                'verb': 'None',
                'verb_class': 0,
                'start_frame': [missing_interval_start],
                'stop_frame': [missing_interval_end],
                'start_timestamp': [missing_interval_start / 22],
                'stop_timestamp': [missing_interval_end / 22],
            })
            dfs.append(new_row)

    if dfs:
        filled_df = pd.concat([df] + dfs, ignore_index=True)
    else:
        filled_df = df.copy()

    return filled_df

def split_rows(data_frame):
    augmented_data = []

    for _, row in data_frame.iterrows():
        start_frame = row['start_frame']
        stop_frame = row['stop_frame']
        interval_size = 110  

        num_intervals = math.ceil((stop_frame - start_frame + 1) / interval_size)

        for i in range(num_intervals):
            new_start = start_frame + i * interval_size
            new_stop = min(new_start + interval_size - 1, stop_frame)  
            new_row = row.copy()
            new_row['start_frame'] = new_start
            new_row['stop_frame'] = new_stop
            new_row['start_timestamp'] = new_start/22
            new_row['stop_timestamp'] = new_stop/22
            augmented_data.append(new_row)

    augmented_dataframe = pd.DataFrame(augmented_data, columns=data_frame.columns)
    augmented_dataframe.reset_index(drop=True, inplace=True)

    return augmented_dataframe

def extend_video_id(df):
    extended_data = []
    num_extensions = 4

    for _ in range(num_extensions):
        extended_data.append(df.copy())

    for i in range(num_extensions):
        extended_data[i]['video_id'] = f'S04_0{i + 2}' 

    result = pd.concat([df] + extended_data, ignore_index=True)
    return result


def create_annotations_file(timestamp_file, annotations_file_test, annotations_file_train):
    import pandas as pd

    activities_to_classify = [
        'Get/replace items from refrigerator/cabinets/drawers',
        'Peel a cucumber',
        'Clear cutting board',
        'Slice a cucumber',
        'Peel a potato',
        'Slice a potato',
        'Slice bread',
        'Spread almond butter on a bread slice',
        'Spread jelly on a bread slice',
        'Open/close a jar of almond butter',
        'Pour water from a pitcher into a glass',
        'Clean a plate with a sponge',
        'Clean a plate with a towel',
        'Clean a pan with a sponge',
        'Clean a pan with a towel',
        'Get items from cabinets: 3 each large/small plates, bowls, mugs, glasses, sets of utensils',
        'Set table: 3 each large/small plates, bowls, mugs, glasses, sets of utensils',
        'Stack on table: 3 each large/small plates, bowls',
        'Load dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils',
        'Unload dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils',
    ]

    video_fps = 22

    timestamps = pd.read_pickle(timestamp_file)
    timestamps = timestamps.drop(
        ['myo_left_timestamps', 'myo_right_timestamps', 'myo_left_readings', 'myo_right_readings'], axis=1)
    timestamps = timestamps.reset_index()

    start_timestamp = timestamps['start'].iloc[0]
    timestamps['start_timestamp'] = timestamps['start'] - start_timestamp
    timestamps['stop_timestamp'] = timestamps['stop'] - start_timestamp
    timestamps['start_frame'] = (timestamps['start_timestamp'] * video_fps).astype(int)
    timestamps['stop_frame'] = (timestamps['stop_timestamp'] * video_fps).astype(int)
    
    data_frame_train = pd.read_pickle(annotations_file_train)
    data_frame_train = data_frame_train[data_frame_train['file'] == 'S04_1.pkl']
    data_frame_train = data_frame_train.drop(['labels'], axis=1)
    data_frame_train['type'] = 'train'

    data_frame_test = pd.read_pickle(annotations_file_test)
    data_frame_test = data_frame_test[data_frame_test['file'] == 'S04_1.pkl']
    data_frame_test = data_frame_test.drop(['labels'], axis=1)
    data_frame_test['type'] = 'test'
    
    
    join_data_frame = pd.concat([data_frame_train, data_frame_test], ignore_index=True)
    join_data_frame.to_csv("JOINED.csv", index=False)

    result = pd.merge(timestamps, join_data_frame, on='index', how='inner')

    result['uid'] = result['index']
    result['participant_id'] = 'S04'
    result['video_id'] = 'S04_01'
    result['verb'] = result['description_x']
    result['narration'] = result['description_x']
    result['verb_class'] = result['description_x'].apply(lambda x: activities_to_classify.index(x))

    result = result[
        ['uid', 'participant_id', 'video_id', 'narration', 'start_timestamp', 'stop_timestamp', 'start_frame',
         'stop_frame', 'verb', 'verb_class', 'type']]
    
    
    #result = fill_missing_intervals(result)
    
    result = result.sort_values(by='start_frame')
    result['uid'] = range(len(result))
    
    train_proportion = 0.7

    none_rows = result[result['type'].isna()]

    num_train = int(len(none_rows) * train_proportion)

    train_indices = none_rows.sample(n=num_train, random_state=42).index
    result.loc[train_indices, 'type'] = 'train'

    test_indices = none_rows.drop(train_indices).index
    result.loc[test_indices, 'type'] = 'test'
    
    result = extend_video_id(result)
    result = split_rows(result)
    result['uid'] = range(len(result))
    
    
    test = result[result['type'] == 'test']
    test.to_csv("S04_test.csv", index=False)
    test.to_pickle("S04_test.pkl")
    train = result[result['type'] == 'train']
    train.to_csv("S04_train.csv", index=False)
    train.to_pickle("S04_train.pkl")
    
    
    #result['uid'] = range(len(result))
    result.to_pickle(f"JOINED.pkl")
    result.to_csv(f"JOINED.csv", index=False)


In [18]:
create_annotations_file('S04_1.pkl', 'action-net/ActionNet_test.pkl', 'action-net/ActionNet_train.pkl')